# <div style="color:white; background:#E606DE; font-size:35px; padding:10px; margin:5px; border-radius:25px; text-align:center">Alzheimer's Diagnosis with Deep Learning</div>

## <div style="color:white; background:#E606DE; font-size:30px; padding:10px; margin:5px; border-radius:25px; text-align:center">Importing Dependencies</div>

In [ ]:
import os
import cv2
import random
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from skimage.io import imread
import matplotlib.pyplot as plt
from matplotlib.image import imread
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization

In [ ]:
# defining training & testing data path
train_data_path='/kaggle/input/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/train'
test_data_path='/kaggle/input/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/test'

## <div style="color:white; background:#E606DE; font-size:30px; padding:10px; margin:5px; border-radius:25px; text-align:center">Visualizing MRI Images</div>

In [ ]:
# Load the images
no_impairment = imread(train_data_path+"/No Impairment"+"/NoImpairment (1).jpg")
mild_impairment = imread(train_data_path+"/Mild Impairment"+"/MildImpairment (1).jpg")
moderate_impairment = imread(train_data_path+"/Moderate Impairment"+"/ModerateImpairment (1).jpg")
very_mild_impairment = imread(train_data_path+"/Very Mild Impairment"+"/VeryMildImpairment (1).jpg")


# Create subplots
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

axes = axes.flatten()

axes[0].imshow(no_impairment, cmap='gray')
axes[0].set_title('No Impairment', fontsize=18)
axes[0].axis('off')

axes[1].imshow(mild_impairment, cmap='gray')
axes[1].set_title('Mild Impairment', fontsize=18)
axes[1].axis('off')

axes[2].imshow(moderate_impairment, cmap='gray')
axes[2].set_title('Moderate Impairment', fontsize=18)
axes[2].axis('off')

axes[3].imshow(very_mild_impairment, cmap='gray')
axes[3].set_title('Very Mild Impairment', fontsize=18)
axes[3].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Data shape
random_image=train_data_path+"/No Impairment"+"/NoImpairment (1).jpg"
print("Image Shape:",cv2.imread(random_image).shape)

## <div style="color:white; background:#E606DE; font-size:30px; padding:10px; margin:5px; border-radius:25px; text-align:center">Data Preprocessing</div>

In [ ]:
def data_preprocessing(output_size, val_split=0.1):
    # ImageDataGenerator for training with validation split
    train_datagen = ImageDataGenerator(rescale=1./255,
                                       zoom_range=0.2,
                                       shear_range=0.2,
                                       rotation_range=15,
                                       fill_mode='nearest',
                                       horizontal_flip=True,
                                       validation_split=val_split)  

    # Training data generator
    training_set = train_datagen.flow_from_directory(train_data_path,
                                                     target_size=(output_size, output_size),
                                                     batch_size=32,
                                                     class_mode='categorical',
                                                     shuffle=True,
                                                     subset='training')  

    # Validation data generator
    validation_set = train_datagen.flow_from_directory(train_data_path,
                                                       target_size=(output_size, output_size),
                                                       batch_size=32,
                                                       class_mode='categorical',
                                                       shuffle=True,
                                                       subset='validation')  

    return training_set, validation_set


# Splitting Data
training_set, validation_set = data_preprocessing(output_size=150, val_split=0.1)

In [ ]:
# Training & Validation data Size
print("Training set distribution:", dict(pd.Series(training_set.classes).value_counts()))
print("\nValidation set distribution:", dict(pd.Series(validation_set.classes).value_counts()))

## <div style="color:white; background:#E606DE; font-size:30px; padding:10px; margin:5px; border-radius:25px; text-align:center">Training the Model</div>

In [ ]:
# Load the pre-trained VGG16 model (excluding the top layers) with weights from ImageNet
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the layers of VGG16 so that they are not trainable
vgg_base.trainable = False

# Initialize the Sequential model
model = Sequential([
    # VGG16 as the base feature extractor
    vgg_base,

    # Flatten the output of the VGG16 base
    Flatten(),

    # Fully Connected Layer 1
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    # Fully Connected Layer 2
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    

    # Output Layer with Softmax Activation (for 4 classes)
    Dense(4, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()

In [ ]:
# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Training the model
history = model.fit(training_set, validation_data = validation_set, batch_size = 64, epochs= 50, verbose = 1)

## <div style="color:white; background:#E606DE; font-size:30px; padding:10px; margin:5px; border-radius:25px; text-align:center">Evaluating Model Performance</div>

In [ ]:
# Model's Performance
sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 5))

# Plotting Loss
axes[0].plot(history.history['loss'], color='#FF6347', label='Training Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], color='#4169E1', label='Validation Loss', linewidth=2, linestyle='--')
axes[0].set_title('Model Loss', fontsize=16, weight='bold')
axes[0].set_xlabel('Epochs', fontsize=12, weight='bold')
axes[0].set_ylabel('Loss', fontsize=12, weight='bold')
axes[0].legend(loc='upper right', fontsize=12)
axes[0].spines['top'].set_visible(False)
axes[0].spines['right'].set_visible(False)

# Plotting Accuracy
axes[1].plot(history.history['accuracy'], color='#FF6347', label='Training Accuracy', linewidth=2)
axes[1].plot(history.history['val_accuracy'], color='#4169E1', label='Validation Accuracy', linewidth=2, linestyle='--')
axes[1].set_title('Model Accuracy', fontsize=16, weight='bold')
axes[1].set_xlabel('Epochs', fontsize=12, weight='bold')
axes[1].set_ylabel('Accuracy', fontsize=12, weight='bold')
axes[1].legend(loc='lower right', fontsize=12)
axes[1].spines['top'].set_visible(False)
axes[1].spines['right'].set_visible(False)


plt.suptitle('Training and Validation Metrics', fontsize=18, weight='bold', color='#333')
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

In [ ]:
# Evaluate model on the test set
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(test_data_path,
                                                 target_size=(150, 150),
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)  

test_loss, test_accuracy = model.evaluate(test_set, verbose=1)

# Print the test accuracy
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

## <div style="color:white; background:#E606DE; font-size:30px; padding:10px; margin:5px; border-radius:25px; text-align:center">Model Predictions</div>

In [ ]:
# Function for making predictions
def prediction(img_path, class_labels):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  

    # Make the prediction
    prediction = model.predict(img_array)

    # Convert the prediction to class labels
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    predicted_class = class_labels[predicted_class_index]

    return predicted_class, prediction[0]

In [ ]:
# Define the folder path and class labels
folder_path = "/kaggle/input/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/test"
class_labels = ["Mild Impairment", "Moderate Impairment", "No Impairment", "Very Mild Impairment"]

# Prepare a list to store file paths and corresponding class names
image_paths = []
class_names = ["Mild Impairment", "Moderate Impairment", "No Impairment", "Very Mild Impairment"]

for class_name in class_names:
    class_folder = os.path.join(folder_path, class_name)
    image_files = os.listdir(class_folder)
    selected_images = random.sample(image_files, 2)  # Randomly select 2 images
    for img_file in selected_images:
        image_paths.append((os.path.join(class_folder, img_file), class_name))

        
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.ravel()  

for i, (img_path, true_class) in enumerate(image_paths):
    predicted_class, probabilities = prediction(img_path, class_labels)
    img = plt.imread(img_path)
    axes[i].imshow(img, cmap='gray')
    axes[i].axis('off')
    axes[i].set_title(f"Pred: {predicted_class}\nTrue: {true_class}", fontsize=14)

plt.tight_layout()
plt.show()